In [ ]:
import pytest
from spannerlib.session import Session
from spannerlib.ast_node_types import RelationDeclaration, AddFact, Query
from spannerlib.primitive_types import DataTypes, Span

Installation NLP failed


In [ ]:
def test_CheckReservedRelationNames()->None:
    with pytest.raises(Exception) as exc_info:
        my_session = Session()
        my_session.run_commands("""
                    new age__spannerlog__(str,int)
                    age__spannerlog__("Allie",23)
                """)
    assert str(exc_info.value) == "encountered relation name: age__spannerlog__. names containing __spannerlog__ are reserved"
    
test_CheckReservedRelationNames()

In [ ]:
def test_FixStrings()->None:
    my_session = Session()
    output = my_session.run_commands("""
                new age(str,int)
                age("\\\nAllie",23)
                age("\\nAllie",23)
                ?age(X,Y)
            """)
    expected_output = r"[(age(X, Y), [('Allie', 23), ('\\nAllie', 23)])]"
    assert str(output) == expected_output
    
test_FixStrings()

printing results for query 'age(X, Y)':
    X    |   Y
---------+-----
  Allie  |  23
 \nAllie |  23



In [ ]:
def test_ConvertSpanNodesToSpanInstances():
    my_session = Session()
    output = my_session.run_commands("""
                new price(str,span)
                price("milk",[5,10))
                ?price(X,Y)
            """)
    relation_name, fact = output[0]
    product_name, product_price_span = fact[0]
    
    expected_product_name = "milk"
    expected_product_price_span = Span(5, 10)
    
    assert product_name == expected_product_name
    assert product_price_span == expected_product_price_span

test_ConvertSpanNodesToSpanInstances()

printing results for query 'price(X, Y)':
  X   |    Y
------+---------
 milk | [5, 10)



In [ ]:
def test_CheckDefinedReferencedVariables():
    test_cases = [
        """
        new sisters(str,str)
        sisters("Sara","Kyra")
        sisters("Allie",f)
        """,
        """
        new sisters(str,str)
        sisters("Sara","Kyra")
        sisters_of_Sara(X) <- sisters(f,X)
        """,
        """
        new sisters(str,str)
        sisters("Sara","Kyra")
        ?sisters(f,X)
        """
    ]
    
    expected_error = 'variable "f" is not defined'
    
    for idx, commands in enumerate(test_cases, start=1):
        with pytest.raises(Exception) as exc_info:
            my_session = Session()
            my_session.run_commands(commands)
        
        assert str(exc_info.value) == expected_error, f"Test case {idx} failed"

test_CheckDefinedReferencedVariables()

In [ ]:
def test_CheckDefinedReferencedVariables4():
    my_session = Session()
    output = my_session.run_commands("""
                    new sisters(str,str)
                    sisters("Sara","Kyra") <- True
                    sisters("Sara","Kyra") <- False
                    ?sisters(X,Y)
                """)
    assert str(output) == '''[(sisters(X, Y), [])]'''
test_CheckDefinedReferencedVariables4()

printing results for query 'sisters(X, Y)':
[]



In [ ]:
def test_CheckReferencedRelationsExistenceAndArity():
    test_cases = [
        (
            """
            new sisters(str, str)
            sisters("Allie")
            """,
            '''relation "sisters" was referenced with an incorrect arity: 1. The correct arity is: 2'''
        ),
        (
            """
            new sisters(str, str)
            sisters("Allie", "Sara")
            brothers_or_sisters(X, Y) <- sisters(X, Y)
            brothers_or_sisters(X, Y) <- brothers(X, Y)
            """,
            '''relation "brothers" is not defined'''
        ),
        (
            """
            new sisters(str, str)
            sisters("Allie", "Sara")
            brothers_or_sisters(X, Y) <- sisters(X, Y)
            ?brothers_or_sisters(X)
            """,
            '''relation "brothers_or_sisters" was referenced with an incorrect arity: 1. The correct arity is: 2'''
        ),
        (
            """
            new sisters(str, str)
            sisters("Allie", "Sara")
            new brothers(str, str)
            brothers("Sam", "Noah")
            brothers_or_sisters(X, Y) <- sisters(X, Y)
            brothers_or_sisters(X, Y) <- brothers(X, Y, Z)
            """,
            '''relation "brothers" was referenced with an incorrect arity: 3. The correct arity is: 2'''
        )
    ]
    
    for idx, (commands, expected) in enumerate(test_cases, start=1):
        with pytest.raises(Exception) as exc_info:
            my_session = Session()
            my_session.run_commands(commands)
        
        assert str(exc_info.value) == expected, f"Test case {idx} failed"

test_CheckReferencedRelationsExistenceAndArity()

In [ ]:
def test_CheckRuleSafety1():
    with pytest.raises(Exception) as exc_info:
        my_session = Session()
        output = my_session.run_commands("""
                    new daughter(str)
                    daughter("Allie")
                    mom(X,Y) <- daughter(Y)
                """)
    expected = '''The rule "mom(X, Y) <- daughter(Y)" \nis not safe because the following free variables appear in the rule head but not as output terms in the rule body:\n{'X'}'''
    assert str(exc_info.value) == expected

test_CheckRuleSafety1()

In [ ]:
def test_TypeCheckRelations1():
    with pytest.raises(Exception) as exc_info:
        my_session = Session()
        output = my_session.run_commands("""
                    new daughter(str)
                    daughter(1)
                """)
    expected = '''type check failed for fact: "daughter(1)"'''
    assert str(exc_info.value) == expected

test_TypeCheckRelations1()

In [ ]:
def test_TypeCheckRelations2():
    with pytest.raises(Exception) as exc_info:
        my_session = Session()
        output = my_session.run_commands("""
                    new age_str(str,str)
                    age_str("Allie","23")
                    new age_int(str,int)
                    age_int("Sam",23)
                    same_age(X)<- age_str(X,Y), age_int(X,Y)
                """)
    expected = '''type check failed for rule "same_age(X) <- age_str(X, Y), age_int(X, Y)"\nbecause the following free variables have conflicting types:\n{'Y'}'''
    assert str(exc_info.value) == expected

test_TypeCheckRelations2()

In [ ]:
def test_SaveDeclaredRelationsSchemas():
    commands = """
                new parent(str, str)
                person(X) <- parent(X, Y)
                person(Y) <- parent(X, Y)
                allie = "Allie"
                jackie = "Jackie"
                parent(jackie, allie)
                ?person(X)
            """
    expected_output = '''[(person(X), [('Allie',), ('Jackie',)])]'''
    
    my_session = Session()
    output = my_session.run_commands(commands)
    
    all_relations = my_session._symbol_table.get_all_relations()
    all_vars = my_session._symbol_table.get_all_variables()
    
    assert (
        len(all_relations) == 2 and
        len(all_vars) == 2 and
        str(output) == expected_output
    )


test_SaveDeclaredRelationsSchemas()

printing results for query 'person(X)':
   X
--------
 Allie
 Jackie



In [ ]:
def test_ExecuteAssignments():
    commands = """
                allie = "Allie"
                jackie = "Jackie"
                a = 1
            """
    expected_variables = [('allie', 'Allie'), ('jackie', 'Jackie'), ('a', 1)]
    
    my_session = Session()
    output = my_session.run_commands(commands)
    all_vars = my_session._symbol_table.get_all_variables()

    assert len(all_vars) == len(expected_variables)

    for var_info, expected_info in zip(all_vars, expected_variables):
        assert var_info[0] == expected_info[0]
        assert var_info[2] == expected_info[1]

test_ExecuteAssignments()

In [ ]:
def test_AddStatementsToNetxParseGraph():
    my_session = Session()
    output = my_session.run_commands("""
                        new parent(str,str)
                        person(X) <- parent(X,Y)
                        person(Y) <- parent(X,Y)
                        allie = "Allie"
                        jackie = "Jackie"
                        parent(jackie,allie)
                        ?person(X)

            """)
    num_of_commands = len(my_session._parse_graph.get_children(my_session._parse_graph._root_id))
    assert num_of_commands == 5

test_AddStatementsToNetxParseGraph()

printing results for query 'person(X)':
   X
--------
 Allie
 Jackie



In [ ]:
def test_AddRulesToTermGraph():
    my_session = Session()
    output = my_session.run_commands("""
                        new parent(str,str)
                        parent("Taylor","Sam")
                        parent("Elijah","Taylor")
                        grandparent(X,Z)<-parent(X,Y) , parent(Y,Z)
                        ?grandparent(X,Y)
                        new sisters(str,str)
                        sisters("Sam","Kristen")
                        sisters("Allie","Jackie")
                        person(X)<-sisters(X,Y)
                        person(X)<-sisters(Y,X)
                        ?person(X)

            """)
    assert (my_session._term_graph.get_children(my_session._term_graph.get_root_id())) == ['grandparent', 'person']

test_AddRulesToTermGraph()

printing results for query 'grandparent(X, Y)':
   X    |  Y
--------+-----
 Elijah | Sam

printing results for query 'person(X)':
    X
---------
  Allie
 Jackie
 Kristen
   Sam

